In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD Ryzen 7 5800H with Radeon Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, znver3)


In [68]:
using CSV
using DataFrames
using DataFramesMeta


In [3]:
;pwd

/home/jiangda/Desktop/data_analysis/Insurance_report/visualization


In [4]:
function remove_chinese_sign(data::DataFrame)
    columns = names(data)
    columns = replace.(columns,r"_\d、" => s"",)
    columns = replace.(columns,r"_（\d）" => s"")
    columns = replace.(columns,r"：" => s":")
    return [Symbol(a) for a in columns] 
end

remove_chinese_sign (generic function with 1 method)

In [5]:
industry_old = CSV.read("../data/industry_old.csv",DataFrame)
cols = remove_chinese_sign(industry_old)
rename!(industry_old,cols)
#Would be better to use rename!(function,dataframe), the function work on one string

,原保险保费收入,原保险保费收入_财产险,原保险保费收入_人身险,原保险保费收入_人身险_寿险
,Float64,Float64,Float64,Float64
1,19059.0,3888.0,15172.0,12005.0
2,16322.0,2953.0,13369.0,10863.0
3,11638.0,1917.0,9721.0,8261.0
4,8500.0,1305.0,7195.0,6272.0
5,3.80166e8,1.07701e8,2.72465e8,2.07229e8
6,3.54195e8,9.70127e7,2.57182e8,1.9664e8
7,3.3032e8,8.8026e7,2.42294e8,1.86e8
8,3.06618e8,8.0034e7,2.26584e8,1.74908e8
9,2.74496e8,7.10607e7,2.03435e8,1.58326e8


In [ ]:
dropmissing(industry_old[!,[:"投资"]])

In [ ]:
industry_old[!,:"投资"] = reduce(vcat,(dropmissing(@select(industry_old,:"投资" = (:"债券" .+ :"股票和证券投资基金") .* 10000)),dropmissing(industry_old[!,[:"投资"]])))[!,1]

In [ ]:
industry_old[!,:year] = parse.(Int64,map(x-> x.captures[1], match.(r"(\d{4})年(.*)保险业经营情况",industry_old.info)))


In [ ]:
industry_old[!,:month] = map(x->x[begin:end-1],replace(map(x-> x.captures[2], match.(r"(\d{4})年(.*)保险业经营情况",industry_old.info)),""=>"1-12月"))

In [ ]:
industry_old

In [ ]:
industry_old[ industry_old[!,:year] .== 2019,:]

In [ ]:
industry_data = CSV.read("../data/industry_data.csv",DataFrame)
cols_data = remove_chinese_sign(industry_data)
rename!(industry_data,cols_data)

In [ ]:
intersect(names(industry_old),names(industry_data))